In [23]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.gridspec as gridspec
import math
import os
from scipy import ndimage
import cv2

In [30]:
image_path ='D:\\POLIMI\\AN2DL\\AN2DL_CH_2\\an2dl2526c2\\preprocessing_results\\train_patches'
mask_path = os.path.join(image_path, 'masks')
out_dir = os.path.join(os.getcwd(), 'temp', 'blurred_results')

os.makedirs(out_dir, exist_ok=True)

print(f"Image path: {image_path}")
print(f"Mask path: {mask_path}")
print(f"Output directory: {out_dir}")

Image path: D:\POLIMI\AN2DL\AN2DL_CH_2\an2dl2526c2\preprocessing_results\train_patches
Mask path: D:\POLIMI\AN2DL\AN2DL_CH_2\an2dl2526c2\preprocessing_results\train_patches\masks
Output directory: d:\POLIMI\AN2DL\AN2DL_CH_2\Scripts\temp\blurred_results


In [36]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

def apply_blur_batch(images_dir, masks_dir, output_dir, blur_strength=(51, 51)):
    """
    Applies the blur mask logic to all images in a directory.
    Assumes mask filenames match image filenames.
    """
    # 1. Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Created output directory: {output_dir}")

    # 2. Iterate through files in the images directory
    supported_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.tiff')
    files = [f for f in os.listdir(images_dir) if f.lower().endswith(supported_extensions)]

    print(f"Found {len(files)} images to process.")

    for filename in tqdm(files, desc="Processing images", unit="img"):
        img_path = os.path.join(images_dir, filename)
        mask_filename = filename.replace('img_', 'mask_', 1) if filename.startswith('img_') else filename
        mask_path = os.path.join(masks_dir, mask_filename)
        output_path = os.path.join(output_dir, filename)

        # 3. Validation
        if not os.path.exists(mask_path):
            print(f"Skipping {filename}: No corresponding mask found at {mask_path}")
            continue

        # 4. Load images
        img = cv2.imread(img_path)
        mask = cv2.imread(mask_path, 0) # Load mask as grayscale

        if img is None or mask is None:
            print(f"Error: Could not load data for {filename}")
            continue

        # 5. Resize mask if dimensions don't match
        if img.shape[:2] != mask.shape:
            # print(f"Resizing mask for {filename}")
            mask = cv2.resize(mask, (img.shape[1], img.shape[0]))

        # 6. Apply Blur Logic
        blurred_img = cv2.GaussianBlur(img, blur_strength, 0)
        
        # Ensure mask is binary: White (255) = Intact, Black (0) = Blurred
        _, binary_mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

        # Region A: Intact (where mask is 255)
        intact_parts = cv2.bitwise_and(img, img, mask=binary_mask)

        # Region B: Blurred (where mask is 0)
        mask_inverted = cv2.bitwise_not(binary_mask)
        blurred_parts = cv2.bitwise_and(blurred_img, blurred_img, mask=mask_inverted)

        # Combine
        result = cv2.add(intact_parts, blurred_parts)

        # 7. Save Result
        cv2.imwrite(output_path, result)


In [37]:
apply_blur_batch(image_path, mask_path, os.path.join(out_dir))

Found 2788 images to process.


Processing images: 100%|██████████| 2788/2788 [01:13<00:00, 37.93img/s]
